In [56]:
import os
import pretty_midi
import numpy as np
from music21 import converter, note, chord
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [57]:
# Fungsi untuk ekstraksi fitur MIDI
def extract_midi_features(filepath):
    try:
        midi_data = pretty_midi.PrettyMIDI(filepath)

        # Distribusi pitch
        pitches = [note.pitch for instrument in midi_data.instruments for note in instrument.notes]
        pitch_mean = np.mean(pitches) if pitches else 0
        pitch_std = np.std(pitches) if pitches else 0

        # Durasi total
        total_duration = midi_data.get_end_time()

        # Jumlah instrumen
        num_instruments = len(midi_data.instruments)

        # Variasi tempo
        tempos = midi_data.get_tempo_changes()[1]
        tempo_mean = np.mean(tempos) if len(tempos) > 0 else 120  # Default ke 120 BPM
        tempo_std = np.std(tempos) if len(tempos) > 0 else 0

        return [pitch_mean, pitch_std, total_duration, num_instruments, tempo_mean, tempo_std]

    except Exception as e:
        print(f"Error dalam memproses {filepath}: {e}")
        return [0, 0, 0, 0, 0, 0]


In [58]:
# Direktori dataset
folder_path = "C:/Users/Public/finalproject/songs"

all_features = []
all_labels = []

# Loop untuk mengekstrak fitur MIDI dari setiap komposer
for composer in ['beeth', 'mozart', 'chopin', 'schubert']:
    composer_folder = os.path.join(folder_path, composer)
    if os.path.exists(composer_folder):
        for file_name in os.listdir(composer_folder):
            if file_name.endswith('.mid') or file_name.endswith('.midi'):
                file_path = os.path.join(composer_folder, file_name)

                # Ekstraksi fitur dari file MIDI
                features = extract_midi_features(file_path)
                if features and any(features): 
                    all_features.append(features)
                    all_labels.append(composer.capitalize())
                else:
                    print(f"Fitur tidak valid untuk file: {file_name}")

print("Ekstraksi dataset selesai.")

Ekstraksi dataset selesai.


In [59]:
# Encode label komposer
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(all_labels)

In [60]:
# Standardisasi fitur
if len(all_features) > 0:
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(np.array(all_features))
else:
    raise ValueError("all_features kosong! Periksa file MIDI atau fungsi ekstraksi.")

In [61]:
# Bagi dataset menjadi train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, encoded_labels, test_size=0.2, random_state=42, stratify=encoded_labels
)

In [62]:
# Model 1: Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_y_pred = rf_model.predict(X_test)
print("\nEvaluasi Model pada Testing Set: Random Forest")
print("Accuracy:", accuracy_score(y_test, rf_y_pred))
print("Classification Report:\n", classification_report(y_test, rf_y_pred, target_names=encoder.classes_))


Evaluasi Model pada Testing Set: Random Forest
Accuracy: 0.6153846153846154
Classification Report:
               precision    recall  f1-score   support

       Beeth       0.50      0.50      0.50         6
      Chopin       0.80      0.80      0.80        10
      Mozart       0.75      0.75      0.75         4
    Schubert       0.33      0.33      0.33         6

    accuracy                           0.62        26
   macro avg       0.60      0.60      0.60        26
weighted avg       0.62      0.62      0.62        26



In [63]:
# Evaluasi model Random Forest pada training set
rf_y_pred_train = rf_model.predict(X_train)
print("\nEvaluasi Model pada Training Set: Random Forest")
print("Accuracy:", accuracy_score(y_train, rf_y_pred_train))
print("Classification Report:\n", classification_report(y_train, rf_y_pred_train, target_names=encoder.classes_))


Evaluasi Model pada Training Set: Random Forest
Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

       Beeth       1.00      1.00      1.00        23
      Chopin       1.00      1.00      1.00        38
      Mozart       1.00      1.00      1.00        16
    Schubert       1.00      1.00      1.00        23

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



In [64]:
# Model 2: K-Nearest Neighbors
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train, y_train)
knn_y_pred = knn_model.predict(X_test)
print("\nEvaluasi Model pada Testing Set: K-Nearest Neighbors")
print("Accuracy:", accuracy_score(y_test, knn_y_pred))
print("Classification Report:\n", classification_report(y_test, knn_y_pred, target_names=encoder.classes_))


Evaluasi Model pada Testing Set: K-Nearest Neighbors
Accuracy: 0.5384615384615384
Classification Report:
               precision    recall  f1-score   support

       Beeth       0.50      0.67      0.57         6
      Chopin       0.56      0.50      0.53        10
      Mozart       0.67      0.50      0.57         4
    Schubert       0.50      0.50      0.50         6

    accuracy                           0.54        26
   macro avg       0.56      0.54      0.54        26
weighted avg       0.55      0.54      0.54        26



In [65]:
knn_y_pred_train = knn_model.predict(X_train)
print("\nEvaluasi Model pada Training Set: K-Nearest Neighbors")
print("Accuracy:", accuracy_score(y_train, knn_y_pred_train))
print("Classification Report:\n", classification_report(y_train, knn_y_pred_train, target_names=encoder.classes_))


Evaluasi Model pada Training Set: K-Nearest Neighbors
Accuracy: 0.73
Classification Report:
               precision    recall  f1-score   support

       Beeth       0.58      0.65      0.61        23
      Chopin       0.72      0.76      0.74        38
      Mozart       1.00      0.94      0.97        16
    Schubert       0.74      0.61      0.67        23

    accuracy                           0.73       100
   macro avg       0.76      0.74      0.75       100
weighted avg       0.74      0.73      0.73       100



In [66]:
# Fungsi untuk memprediksi komposer berdasarkan file MIDI
def predict_composer(midi_file, model, encoder, scaler):
    # Ekstraksi fitur dari file MIDI
    features = extract_midi_features(midi_file)

    # Validasi jika fitur tidak valid
    if not features or not any(features):
        raise ValueError(f"Fitur tidak valid untuk file: {midi_file}")

    # Standarisasi fitur
    features_scaled = scaler.transform([features])

    # Prediksi menggunakan model
    predicted_label = model.predict(features_scaled)

    # Mengembalikan hasil prediksi dalam nama komposer
    predicted_composer = encoder.inverse_transform(predicted_label)
    return predicted_composer[0]

In [67]:
# Path ke file MIDI yang ingin diprediksi
midi_file = "C:/Users/Public/finalproject/songs/Losing True (Album Version).mid" 

In [68]:
# Prediksi komposer
predicted_composer = predict_composer(midi_file, rf_model, encoder, scaler)
print(f"Komposer yang diprediksi untuk {midi_file} adalah: {predicted_composer}")

Komposer yang diprediksi untuk C:/Users/Public/finalproject/songs/Losing True (Album Version).mid adalah: Chopin


In [69]:
# Evaluasi model Random Forest pada test set
rf_y_pred = rf_model.predict(X_test)
print("\nEvaluasi Model: Random Forest")
print("Accuracy:", accuracy_score(y_test, rf_y_pred))
print("Classification Report:\n", classification_report(y_test, rf_y_pred, target_names=encoder.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_test, rf_y_pred))

# Evaluasi model K-Nearest Neighbors pada test set
knn_y_pred = knn_model.predict(X_test)
print("\nEvaluasi Model: K-Nearest Neighbors")
print("Accuracy:", accuracy_score(y_test, knn_y_pred))
print("Classification Report:\n", classification_report(y_test, knn_y_pred, target_names=encoder.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_test, knn_y_pred))

# Evaluasi prediksi pada data baru (midi_file)
predicted_composer = predict_composer(midi_file, rf_model, encoder, scaler)
print(f"Komposer yang diprediksi untuk {midi_file} adalah: {predicted_composer}")


Evaluasi Model: Random Forest
Accuracy: 0.6153846153846154
Classification Report:
               precision    recall  f1-score   support

       Beeth       0.50      0.50      0.50         6
      Chopin       0.80      0.80      0.80        10
      Mozart       0.75      0.75      0.75         4
    Schubert       0.33      0.33      0.33         6

    accuracy                           0.62        26
   macro avg       0.60      0.60      0.60        26
weighted avg       0.62      0.62      0.62        26

Confusion Matrix:
 [[3 0 0 3]
 [1 8 0 1]
 [0 1 3 0]
 [2 1 1 2]]

Evaluasi Model: K-Nearest Neighbors
Accuracy: 0.5384615384615384
Classification Report:
               precision    recall  f1-score   support

       Beeth       0.50      0.67      0.57         6
      Chopin       0.56      0.50      0.53        10
      Mozart       0.67      0.50      0.57         4
    Schubert       0.50      0.50      0.50         6

    accuracy                           0.54        26
 